Import libs, reading the csv

In [4]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None

In [5]:
df=pd.read_csv("country_vaccination_stats.csv")

Understanding the data

In [6]:
df.shape

(1502, 4)

In [7]:
df.head()

,country,date,daily_vaccinations,vaccines
0,Argentina,12/29/2020,NaN,Sputnik V
1,Argentina,12/30/2020,15656.0,Sputnik V
2,Argentina,12/31/2020,15656.0,Sputnik V
3,Argentina,1/1/2021,11070.0,Sputnik V
4,Argentina,1/2/2021,8776.0,Sputnik V


In [8]:
len(df["country"].unique())

60

In [9]:
df.isnull().sum()

country                0
date                   0
daily_vaccinations    60
vaccines               0
dtype: int64

Question 4:

Code Implementation Task: Implement code to fill the missing data (impute) in daily_vaccinations column per country with the minimum daily vaccination number of relevant countries.  
Note: If a country does not have any valid vaccination number yet, fill it with “0” (zero). 
Please  provide the link to your code as answer to this question.

In [10]:
# Create a new DataFrame to hold the minimum daily vaccination numbers per country
min_daily_vaccinations = df.groupby('country')['daily_vaccinations'].min().reset_index()

In [11]:
# Checking if any null values for a country.
min_daily_vaccinations.isnull().sum()

country               0
daily_vaccinations    1
dtype: int64

In [12]:
#Finding the country which has the null value.
min_daily_vaccinations.loc[min_daily_vaccinations["daily_vaccinations"].isnull()==True]

,country,daily_vaccinations
30,Kuwait,NaN


In [13]:
#Filling the missing daily vaccination for the country with 0.(Kuwait)
min_daily_vaccinations['daily_vaccinations'] = min_daily_vaccinations['daily_vaccinations'].fillna(0)

In [14]:
#Checking if fixed
min_daily_vaccinations.isnull().sum()

country               0
daily_vaccinations    0
dtype: int64

In [15]:
# Merge the minimum daily vaccination numbers back into the original DataFrame
df2 = df.merge(min_daily_vaccinations, on='country', suffixes=('', '_min'))

In [16]:
#Filling the missing values with daily_vaccinations_min.
df2["daily_vaccinations"]=df2["daily_vaccinations"].fillna(df2["daily_vaccinations_min"])

In [17]:
# Drop the temporary minimum daily vaccinations column
df2 = df2.drop('daily_vaccinations_min', axis=1)

In [18]:
df2.head()

,country,date,daily_vaccinations,vaccines
0,Argentina,12/29/2020,6483.0,Sputnik V
1,Argentina,12/30/2020,15656.0,Sputnik V
2,Argentina,12/31/2020,15656.0,Sputnik V
3,Argentina,1/1/2021,11070.0,Sputnik V
4,Argentina,1/2/2021,8776.0,Sputnik V


In [19]:
#Check if solved
df2.isnull().sum()

country               0
date                  0
daily_vaccinations    0
vaccines              0
dtype: int64

Question 5:

Code Implementation Task: Implement code to list the top-3 countries with highest median daily vaccination numbers by considering missing values imputed version of dataset.
Please  provide the link to your code as answer to this question.

In [20]:
# Compute the median daily vaccination number for each country
median_daily_vaccinations = df2.groupby('country')['daily_vaccinations'].median().reset_index()


In [21]:
#Check
median_daily_vaccinations.head()

,country,daily_vaccinations
0,Argentina,11645.0
1,Austria,7850.0
2,Bahrain,3662.0
3,Belgium,3606.0
4,Brazil,17220.0


In [22]:
# Sort the median daily vaccination numbers in descending order and select the top-3 countries
top_3_countries = median_daily_vaccinations.sort_values(by='daily_vaccinations', ascending=False).head(3)

In [23]:
#Check
top_3_countries

,country,daily_vaccinations
58,United States,378253.0
8,China,276786.0
24,India,173922.0


In [24]:
# Print the top-3 countries as a list.
print(top_3_countries['country'].tolist())

['United States', 'China', 'India']


Question 6:

What is the number of total vaccinations done on 1/6/2021 (MM/DD/YYYY) by considering missing values imputed version of dataset? Please  just provide the number as answer.

In [25]:
#Filtering the number by using loc function.
result=df2["daily_vaccinations"].loc[df2['date'] == '1/6/2021'].sum()

In [26]:
#Printing the result
print(result)

1485255.0


In [27]:
print(df2.columns)

Index(['country', 'date', 'daily_vaccinations', 'vaccines'], dtype='object')


Question 7:

Code Implementation Task: If this list would be a database table, please provide SQL query to fill in the missing daily vaccination numbers with discrete median of country as similar to question a.  
Please  provide the link to your code as answer to this question. 
Note: This time SQL equivalent is requested, and imputation value is median of each country, not minimum. Please remember filling countries with zero if they do not have any valid daily_vaccination records like Kuwait

In [28]:
""" UPDATE vaccinations
SET daily_vaccinations = (
  SELECT COALESCE(MEDIAN(daily_vaccinations), 0)
  FROM vaccinations v2
  WHERE v2.country = vaccinations.country AND daily_vaccinations IS NOT NULL
)
WHERE daily_vaccinations IS NULL; """

' UPDATE vaccinations\nSET daily_vaccinations = (\n  SELECT COALESCE(MEDIAN(daily_vaccinations), 0)\n  FROM vaccinations v2\n  WHERE v2.country = vaccinations.country AND daily_vaccinations IS NOT NULL\n)\nWHERE daily_vaccinations IS NULL; '